In [1]:
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook
import os

In [2]:
PATH_FILES_ONE_DRIVE = Path(r'C:\Users\h-pgy\one_drive_prefs\OneDrive - Default Directory\Shared Documents\Estruturação do PDM 2021-2024\Elaboração PDM Versão Final')

In [3]:
class PlanilhaNotFound(ValueError):
    '''Raised quando nao consegue achar planilha para a meta'''

In [4]:
def listar_metas_ok():
    
    controle = pd.read_excel(PATH_FILES_ONE_DRIVE/'Controle das Devolutivas.xlsx')
    
    filtro_preench = controle['Aba Orçamento preenchida corretamente?'].str.lower()=='sim'
    filtro_analis = controle['Aba orçamento já analisada?'].str.lower()=='sim'
    
    return controle[filtro_preench&filtro_analis]['Meta'].values

In [5]:
def get_planilhas(folder_original):
        
        fpaths =  [os.path.join(folder_original, file) for file in 
                    os.listdir(folder_original) if 'xls' 
                    in file.split('.')[-1]]
        
        return fpaths

In [6]:
def read_workbook(file):

    wb = load_workbook(file, read_only=True, data_only=True)

    return wb

In [7]:
def match_planilha(wb, num_meta):
        
        sheet = wb['Ficha técnica Meta ']
        
        num_meta_planilha = sheet['C5'].value
        
        if str(num_meta_planilha) == str(num_meta):
            return True

In [8]:
def find_planilha(files, num_meta):
    
    for file in files:
        
        wb = read_workbook(file)
        
        if match_planilha(wb, num_meta):
            secretaria = wb['Ficha técnica Meta ']['C4'].value
            return file, secretaria
    else:
        raise PlanilhaNotFound(f'Planilha não encontrada para a meta {num_meta}')

In [9]:
def planilhao(lista_ok = None, path_xls = None, lista_dfs = None):
    
    if path_xls is None:
        path_xls = PATH_FILES_ONE_DRIVE/'Fichas Metas\Devolutiva 11-jun'
        
    if lista_ok is None:
        lista_ok = listar_metas_ok()
        
    if lista_dfs is None:
        lista_dfs = []
    
    f_paths = get_planilhas(path_xls)
        
    for meta in lista_ok:
        print(f'Puxando dados meta {meta}')
        try:
            file, secretaria = find_planilha(f_paths, meta)
        except PlanilhaNotFound as e:
            print(e)
            continue
        df = pd.read_excel(file, sheet_name = 'Orçamento', thousands = '.',
                          dtype = {'Código Ação' : str})
        df['meta'] = meta
        df['secretaria'] = secretaria
        lista_dfs.append(df)
    
    return pd.concat(lista_dfs)
    

In [10]:
lista_dfs = []

In [ ]:
df = planilhao(lista_dfs = lista_dfs)

Puxando dados meta 9
Puxando dados meta 13
Puxando dados meta 14
Puxando dados meta 15
Puxando dados meta 16
Puxando dados meta 18
Puxando dados meta 19
Puxando dados meta 20
Puxando dados meta 21
Puxando dados meta 22
Puxando dados meta 23
Puxando dados meta 24
Puxando dados meta 25
Puxando dados meta 26
Puxando dados meta 27
Puxando dados meta 28


In [ ]:
def dropar_unnamed(df):
    
    unnameds = [col for col in df if
               col.startswith('Unnamed: ')]
    
    df.drop(unnameds, axis = 1, inplace = True)
    for col in df:
        if col.startswith('Unnamed: '):
            df.drop(col, axis = 1, )

In [ ]:
dropar_unnamed(df)

In [ ]:
sem_na = df.dropna(how='all', subset = [col for col in df.columns if col not in ('meta', 'secretaria')]).copy()

In [ ]:
def pegar_objetivos(df):
    
    objetivos = pd.read_excel('original_data/Metas com Objetivos atualizados 18-06-21.xlsx')
    
    merged = df.merge(objetivos, how = 'left', 
                      right_on = 'Nº meta PdM ',  left_on = 'meta').copy()
    
    merged.drop(['Redação da Meta PdM ','Nº meta PdM '],
               axis = 1, inplace = True)
    
    return merged

In [ ]:
sem_na = pegar_objetivos(sem_na)

In [ ]:
'XX.XX.XXXX.XX'.split('.')

In [ ]:
def add_ponto_acao(item):
    
    item = str(item)
    
    if item == 'nan':
        return 'XX.XX.XXXX.XX'
    else:
        return f'{item[:2]}.{item[2:4]}.{item[4:8]}.{item[8:10]}'

In [ ]:
sem_na['Código Ação'] = sem_na['Código Ação'].apply(add_ponto_acao)

In [ ]:
sem_na['Unidade Orçamentária'] = sem_na['Código Ação'].apply(lambda x: '.'.join(x.split('.')[:2]))

In [ ]:
sem_na['Ação'] = sem_na['Código Ação'].apply(lambda x: x.split('.')[2])

In [ ]:
sem_na['Fonte'] = sem_na['Código Ação'].apply(lambda x: x.split('.')[3])

In [ ]:
ordem_cols = [
    'Objetivo Estratégico Atualizado',
    'secretaria',
    'meta',
    'IN',
    'Item',
    'Custeio/\nInvestimento',
    'Código Ação',
    'Unidade Orçamentária',
    'Ação',
    'Fonte',
    'classif.',
    'V. Atualizado',
    'V. Congelado',
    'V. Disponível',
    'Atualizado (Disponibilizado PDM)',
    'Congelado (Dispon. PDM)',
    'Disponível (Dispon. PDM)',
    'Custo 2021',
    'Custo 2022',
    'Custo 2023',
    'Custo 2024',
    'Custo TOTAL',
     
     ]

In [ ]:
sem_na = sem_na[ordem_cols]

In [ ]:
sem_na.to_excel('teste_planilhao_orcamento_2.xlsx', index = False)

In [ ]:
colunas_numericas = [
'V. Atualizado',
'V. Congelado',                       
'V. Disponível',                        
'Atualizado (Disponibilizado PDM)',
'Congelado (Dispon. PDM)',    
'Disponível (Dispon. PDM)',             
'Custo 2021',            
'Custo 2022',                          
'Custo 2023',                          
'Custo 2024',                          
'Custo TOTAL']

In [ ]:
def solve_numero(item):
    
    if type(item) in (float, int):
        return item
    elif type(item) is str:
        print(item)
        try:
            float(item)
        except ValueError:
            try:
                item = item.replace('.', '')
                item = item.replace(',', '.')
                return float(item)
            except ValueError:
                print(f'Item {item} fora do padrão')
                return 0
    else:
        print(type(item))

In [ ]:
for col in colunas_numericas:
    sem_na[col] = sem_na[col].apply(solve_numero)

In [ ]:
sem_na.dtypes

In [ ]:
df[df.eq('513,150,00').any(1)]

In [ ]:
n_pode_vazio = [
    'Atualizado (Disponibilizado PDM)',
    'Congelado (Dispon. PDM)',
    'Disponível (Dispon. PDM)',
    'Custo 2021',
    'Custo 2022',
    'Custo 2022',
    'Custo 2023',
    'Custo 2024'
]

In [ ]:
for col in n_pode_vazio:
    soma = sem_na[col].isnull().sum()
    if  soma > 0:
        print(col, soma)

In [ ]:
dropar = (sem_na['classif.'].isnull()|sem_na['Custeio/\nInvestimento'].isnull()|sem_na['Objetivo Estratégico Atualizado'].isnull())

In [ ]:
sem_na[sem_na.isnull()]